<a href="https://colab.research.google.com/github/DinurakshanRavichandran/Visio-Glance/blob/FINAL-MODEL-NLP/unified_eye_disease_detection_corrected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unified Eye Disease Detection Model
This notebook implements a machine learning pipeline to predict one of six eye diseases based on symptom datasets.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


01) Concatenating the 6 datasets.

In [2]:
import pandas as pd

# Load datasets
glaucoma = pd.read_csv('/content/drive/MyDrive/PROJECT 29/FINAL MODEL/preprocessed_glaucoma_dataset.csv')
cataract = pd.read_csv('/content/drive/MyDrive/PROJECT 29/FINAL MODEL/Preprocessed_Cataract_Dataset.csv')
diabetic_retinopathy = pd.read_csv('/content/drive/MyDrive/PROJECT 29/FINAL MODEL/Preprocessed_Diabetic_Retinopathy_Dataset.csv')
cnv = pd.read_csv('/content/drive/MyDrive/PROJECT 29/FINAL MODEL/Preprocessed_CNV_Detection_Dataset.csv')
dme = pd.read_csv('/content/drive/MyDrive/PROJECT 29/FINAL MODEL/Preprocessed_DME_Dataset.csv')
drusen = pd.read_csv('/content/drive/MyDrive/PROJECT 29/FINAL MODEL/Preprocessed_Drusen_Dataset.csv')

# Assign diagnosis labels
glaucoma['Diagnosis'] = 'Glaucoma'
cataract['Diagnosis'] = 'Cataract'
diabetic_retinopathy['Diagnosis'] = 'Diabetic Retinopathy'
cnv['Diagnosis'] = 'CNV'
dme['Diagnosis'] = 'DME'
drusen['Diagnosis'] = 'Drusen'

# Concatenate datasets
all_datasets = pd.concat([glaucoma, cataract, diabetic_retinopathy, cnv, dme, drusen], ignore_index=True)
all_datasets.head()


,Age,Intraocular Pressure (IOP),Cup-to-Disc Ratio (CDR),Family History,Visual Field Test Results,Optical Coherence Tomography (OCT) Results,Pachymetry,Visual Symptoms,Diagnosis,RNFL Thickness,...,Hemorrhages Count,Cholesterol_HDL_Ratio,Smoking Status_0,Smoking Status_1,Fluorescein Angiography Results,Visual Acuity Test Results,Lens Status,BMI,Blood Pressure,Cholesterol Levels
0,69,0.448238,-0.889954,0.0,"Sensitivity: 0.54, Specificity: 0.75","RNFL Thickness: 86.48 µm, GCC Thickness: 64.14...",-0.284554,tunnel vision eye pain nausea,Glaucoma,-0.132907,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,69,0.202593,1.188779,0.0,"Sensitivity: 0.72, Specificity: 0.88","RNFL Thickness: 96.88 µm, GCC Thickness: 56.48...",0.105048,redness in the eye vision loss tunnel vision,Glaucoma,1.316332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,67,1.410156,1.188779,0.0,"Sensitivity: 0.56, Specificity: 0.8","RNFL Thickness: 89.81 µm, GCC Thickness: 59.05...",0.827507,halos around lights vision loss redness in the...,Glaucoma,0.331128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23,0.122242,0.426577,0.0,"Sensitivity: 0.6, Specificity: 0.93","RNFL Thickness: 87.25 µm, GCC Thickness: 63.98...",1.416408,nausea nausea halos around lights,Glaucoma,-0.025608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21,-0.375935,-1.721448,0.0,"Sensitivity: 0.82, Specificity: 0.9","RNFL Thickness: 82.61 µm, GCC Thickness: 66.01...",1.338211,eye pain eye pain tunnel vision,Glaucoma,-0.672191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
